In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.stattools import acf
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from xgboost import XGBRegressor
import holidays
import datetime
from datetime import timedelta
import joblib

import warnings
warnings.filterwarnings('ignore')

In [2]:
sales = pd.read_csv('D:/Portfolio/Smart_Restock_Advisor/data/sales.csv', sep=';')
sales = sales.rename(columns={'Номенклатура': 'SKU', 'Покупатель': 'Client', 'Сумма': 'Earned', 'Количество': 'Value'})
sales.sort_values('Date', axis=0, inplace=True)
sales.reset_index(drop=True)
sales['Date'] = pd.to_datetime(sales['Date']).dt.normalize()
#sales = sales.set_index('Date')

In [3]:
sales_with_group = sales.copy()

# Сначала определяем группу товара по средним продажам
sku_avg = sales.groupby('SKU')['Value'].mean()
sku_group_map = pd.cut(sku_avg, bins=[0, 10, 100, float('inf')], 
                       labels=['low', 'medium', 'high'])
sales_with_group['SKU_Group'] = sales_with_group['SKU'].map(sku_group_map)

# Затем определяем группу строки по фактическому значению
def assign_row_group(row):
    if row['Value'] <= 10:
        return 'low'
    elif row['Value'] <= 100:
        return 'medium'
    else:
        return 'high'

sales_with_group['Row_Group'] = sales_with_group.apply(assign_row_group, axis=1)

# Разделяем по Row_Group
low_sales = sales_with_group[sales_with_group['Row_Group'] == 'low']
medium_sales = sales_with_group[sales_with_group['Row_Group'] == 'medium']
high_sales = sales_with_group[sales_with_group['Row_Group'] == 'high']

low_sales = low_sales.drop(['Row_Group', 'SKU_Group'], axis=1)
medium_sales = medium_sales.drop(['Row_Group', 'SKU_Group'], axis=1)
high_sales = high_sales.drop(['Row_Group', 'SKU_Group'], axis=1)

In [4]:
class FeatureEngineer(BaseEstimator, TransformerMixin):
    def __init__(self, date_col='Date', sku_col='SKU', target_col='Value'):
        self.date_col = date_col
        self.sku_col = sku_col
        self.target_col = target_col
        self.required_columns = None
        self.feature_names_ = None
        self.ru_holidays = None
        self.sku_stats = None
        
    def fit(self, X, y=None):
        # X должен быть DataFrame
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
            
        # Запоминаем, какие колонки были на входе
        self.required_columns = X.columns.tolist()
        
        # Проверяем наличие обязательных колонок
        required = [self.date_col, self.sku_col]
        missing = [col for col in required if col not in X.columns]
        if missing:
            raise ValueError(f"Отсутствуют обязательные колонки: {missing}")
        
        # Создаем временный датафрейм для определения выходных фич
        X_temp = X.copy()
        
        # Если есть y, добавляем его для создания лаговых фич
        if y is not None:
            X_temp[self.target_col] = y.values if hasattr(y, 'values') else y
            
        # Создаем фичи на маленькой выборке
        X_sample = X_temp.head(3) if len(X_temp) >= 3 else X_temp
        X_transformed = self._create_time_series_features(X_sample)
        
        # Запоминаем имена фич на выходе
        self.feature_names_ = X_transformed.columns.tolist()
        
        # Вычисляем holidays для всех лет в данных
        if self.date_col in X.columns:
            X_temp[self.date_col] = pd.to_datetime(X_temp[self.date_col])
            years = X_temp[self.date_col].dt.year.unique()
            self.ru_holidays = {}
            for year in years:
                self.ru_holidays.update(holidays.RU(years=int(year)))
        
        # Вычисляем статистики по SKU если есть target
        if y is not None and len(X_temp) > 0:
            X_temp[self.target_col] = y.values if hasattr(y, 'values') else y
            self.sku_stats = X_temp.groupby(self.sku_col)[self.target_col].agg(
                ['mean', 'std', 'median', 'min', 'max']
            ).add_prefix('sku_')
        
        return self
    
    def transform(self, X, y=None):
        # Проверяем наличие всех нужных колонок
        if self.required_columns:
            missing_cols = set(self.required_columns) - set(X.columns)
            if missing_cols:
                raise ValueError(f"Отсутствуют колонки: {missing_cols}")
        
        # Создаем копию данных
        X_transformed = X.copy()
        
        # Если есть y, добавляем его для лаговых фич
        if y is not None:
            # Проверяем совпадение индексов
            if len(X_transformed) != len(y):
                raise ValueError(f"Размеры X ({len(X_transformed)}) и y ({len(y)}) не совпадают")
            X_transformed[self.target_col] = y.values if hasattr(y, 'values') else y
        
        # Создаем все фичи
        X_transformed = self._create_time_series_features(X_transformed)
        
        # Если target_col был добавлен временно, удаляем его
        if y is not None and self.target_col in X_transformed.columns and self.target_col not in X.columns:
            X_transformed = X_transformed.drop(columns=[self.target_col])
        
        return X_transformed
    
    def _create_time_series_features(self, df):
        df = df.copy()
        
        # Сортируем по SKU и дате для правильного расчета лагов
        df = df.sort_values([self.sku_col, self.date_col])
        
        # Преобразуем дату
        df[self.date_col] = pd.to_datetime(df[self.date_col])
        
        # Создаем лаги только если есть target_col
        if self.target_col in df.columns:
            lag_periods = [1, 2, 3, 4, 5, 6, 7, 14, 21, 28, 30, 60, 90]
            for lag in lag_periods:
                df[f'lag_{lag}'] = df.groupby(self.sku_col)[self.target_col].shift(lag)
            
            # Разницы
            for lag in [1, 7, 14, 28]:
                df[f'diff_{lag}'] = df.groupby(self.sku_col)[self.target_col].diff(lag)
            
            # Процентные изменения
            for lag in [1, 7, 28]:
                df[f'pct_change_{lag}'] = df.groupby(self.sku_col)[self.target_col].pct_change(lag).fillna(0)
            
            # Значения со сдвигом для rolling features
            shifted_values = df.groupby(self.sku_col)[self.target_col].shift(1)
            
            # Rolling статистики
            window_sizes = [3, 7, 14, 21, 30, 60]
            for window in window_sizes:
                df[f'rolling_mean_{window}'] = shifted_values.groupby(df[self.sku_col]).rolling(window, min_periods=1).mean().reset_index(level=0, drop=True)
                df[f'rolling_std_{window}'] = shifted_values.groupby(df[self.sku_col]).rolling(window, min_periods=1).std().reset_index(level=0, drop=True)
                df[f'rolling_min_{window}'] = shifted_values.groupby(df[self.sku_col]).rolling(window, min_periods=1).min().reset_index(level=0, drop=True)
                df[f'rolling_max_{window}'] = shifted_values.groupby(df[self.sku_col]).rolling(window, min_periods=1).max().reset_index(level=0, drop=True)
                df[f'rolling_median_{window}'] = shifted_values.groupby(df[self.sku_col]).rolling(window, min_periods=1).median().reset_index(level=0, drop=True)
            
            # EWMA
            for span in [3, 7, 14, 30]:
                df[f'ewma_{span}'] = shifted_values.groupby(df[self.sku_col]).ewm(span=span, adjust=False).mean().reset_index(level=0, drop=True)
        
        # Фичи из даты
        df['year'] = df[self.date_col].dt.year
        df['month'] = df[self.date_col].dt.month
        df['quarter'] = df[self.date_col].dt.quarter
        df['day_of_month'] = df[self.date_col].dt.day
        df['day_of_week'] = df[self.date_col].dt.dayofweek
        df['week_of_year'] = df[self.date_col].dt.isocalendar().week
        df['day_of_year'] = df[self.date_col].dt.dayofyear
        
        # Циклические фичи
        df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
        df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
        df['day_of_week_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
        df['day_of_week_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
        df['day_of_year_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365)
        df['day_of_year_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 365)
        
        # Бинарные фичи
        df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
        df['is_month_start'] = df[self.date_col].dt.is_month_start.astype(int)
        df['is_month_end'] = df[self.date_col].dt.is_month_end.astype(int)
        df['is_quarter_start'] = df[self.date_col].dt.is_quarter_start.astype(int)
        df['is_quarter_end'] = df[self.date_col].dt.is_quarter_end.astype(int)
        df['is_year_start'] = df[self.date_col].dt.is_year_start.astype(int)
        df['is_year_end'] = df[self.date_col].dt.is_year_end.astype(int)
        
        # Сезоны
        def get_season(month):
            if month in [12, 1, 2]:
                return 'winter'
            elif month in [3, 4, 5]:
                return 'spring'
            elif month in [6, 7, 8]:
                return 'summer'
            else:
                return 'autumn'
        
        df['season'] = df['month'].apply(get_season)
        
        # Праздники (используем предварительно вычисленные)
        if self.ru_holidays:
            df['is_holiday'] = df[self.date_col].dt.date.isin(self.ru_holidays).astype(int)
            
            for days_before in [1, 2, 3]:
                holiday_dates = [d - timedelta(days=days_before) for d in self.ru_holidays.keys()]
                df[f'is_{days_before}_days_before_holiday'] = df[self.date_col].dt.date.isin(holiday_dates).astype(int)
            
            for days_after in [1, 2, 3]:
                holiday_dates = [d + timedelta(days=days_after) for d in self.ru_holidays.keys()]
                df[f'is_{days_after}_days_after_holiday'] = df[self.date_col].dt.date.isin(holiday_dates).astype(int)
        
        # Взаимодействия и отношения (только если есть лаги)
        if self.target_col in df.columns:
            df['lag_ratio_7_1'] = df['lag_7'] / (df['lag_1'] + 1e-6)
            df['lag_ratio_14_7'] = df['lag_14'] / (df['lag_7'] + 1e-6)
            df['lag_ratio_28_7'] = df['lag_28'] / (df['lag_7'] + 1e-6)
            df['lag1_to_mean7'] = df['lag_1'] / (df['rolling_mean_7'] + 1e-6)
            df['lag1_to_mean30'] = df['lag_1'] / (df['rolling_mean_30'] + 1e-6)
            
            for lag in [1, 7, 14, 28]:
                df[f'lag_{lag}_zscore'] = (df[f'lag_{lag}'] - df['rolling_mean_30']) / (df['rolling_std_30'] + 1e-6)
        
        # Статистики по SKU (используем предварительно вычисленные или вычисляем на лету)
        if self.sku_stats is not None and self.sku_col in df.columns:
            df = df.merge(self.sku_stats, on=self.sku_col, how='left')
        elif self.target_col in df.columns and self.sku_col in df.columns:
            sku_stats = df.groupby(self.sku_col)[self.target_col].agg(['mean', 'std', 'median', 'min', 'max']).add_prefix('sku_')
            df = df.merge(sku_stats, on=self.sku_col, how='left')
        
        if self.target_col in df.columns:
            df['value_norm'] = (df[self.target_col] - df['sku_mean']) / (df['sku_std'] + 1e-6)
            df['value_percentile'] = df.groupby(self.sku_col)[self.target_col].rank(pct=True)
        
        # Индекс времени
        df['time_index'] = df.groupby(self.sku_col).cumcount()
        
        # Тренды (только если есть target)
        if self.target_col in df.columns:
            for window in [7, 14, 30]:
                df[f'trend_slope_{window}'] = df.groupby(self.sku_col)[self.target_col].shift(1).rolling(
                    window, min_periods=3
                ).apply(lambda x: np.polyfit(range(len(x)), x, 1)[0] if len(x) >= 3 else np.nan).reset_index(level=0, drop=True)
        
        # Статистики по дням недели
        if self.target_col in df.columns:
            for dow in range(7):
                mask = df['day_of_week'] == dow
                if mask.any():
                    dow_stats = df.loc[mask].groupby(self.sku_col)[self.target_col].shift(1).rolling(
                        30, min_periods=5
                    ).mean().reset_index(level=0, drop=True)
                    df.loc[mask, f'dow_{dow}_mean'] = dow_stats
        
        # Аномалии и выбросы (только если есть target)
        if self.target_col in df.columns and 'rolling_std_30' in df.columns:
            df['z_score'] = (df[self.target_col] - df['rolling_mean_30']) / (df['rolling_std_30'] + 1e-6)
            df['is_outlier'] = (df['z_score'].abs() > 3).astype(int)
            
            if 'diff_1' in df.columns and 'rolling_std_7' in df.columns:
                df['spike_flag'] = (df['diff_1'].abs() > 3 * df['rolling_std_7']).astype(int)
        
        # Взаимодействия
        df['month_start_weekend'] = df['is_month_start'] * df['is_weekend']
        df['month_end_weekend'] = df['is_month_end'] * df['is_weekend']
        
        # Заполнение пропусков в лагах
        lag_cols = [col for col in df.columns if col.startswith('lag_')]
        for col in lag_cols:
            df[col] = df.groupby(self.sku_col)[col].transform(lambda x: x.fillna(x.median() if not x.isnull().all() else 0))
        
        # Заполнение пропусков в rolling фичах
        rolling_cols = [col for col in df.columns if col.startswith('rolling_') or col.startswith('ewma_')]
        for col in rolling_cols:
            df[col] = df[col].fillna(df[col].mean() if not df[col].isnull().all() else 0)
        
        # Удаляем временные колонки
        cols_to_drop = ['z_score', 'day_of_year']
        for col in cols_to_drop:
            if col in df.columns:
                df = df.drop(columns=[col])
        
        # Удаляем исходные колонки, которые не должны попасть в модель
        cols_to_keep = [col for col in df.columns if col not in [self.date_col, self.sku_col]]
        if self.target_col in df.columns and self.target_col not in self.required_columns:
            cols_to_keep = [col for col in cols_to_keep if col != self.target_col]
        
        return df[cols_to_keep]
    
    def get_feature_names_out(self, input_features=None):
        return self.feature_names_ if self.feature_names_ is not None else []

In [5]:
def data_preprocessing(df, test_size, target_col='Value'):
    # 1. Подготовка данных
    df_copy = df.reset_index()
    df_copy = df_copy.sort_values(['SKU', 'Date'])
    
    # 2. Разделение на признаки и цель
    X = df_copy.drop([target_col, 'Earned', 'Client'], axis=1)
    y = df_copy[target_col]

    # 3. Разделение на train/test
    #Определяем дату разделения
    unique_dates = df_copy['Date'].sort_values().unique()
    split_idx = int(len(unique_dates) * (1 - test_size))
    split_date = unique_dates[split_idx]
    
    # Разделяем данные
    train_mask = df_copy['Date'] < split_date
    test_mask = df_copy['Date'] >= split_date
    
    X_train = X[train_mask]
    X_test = X[test_mask]
    y_train = y[train_mask]
    y_test = y[test_mask]
    
    # Проверяем SKU
    sku_train = set(X_train['SKU']) if 'SKU' in X_train.columns else set()
    sku_test = set(X_test['SKU']) if 'SKU' in X_test.columns else set()
    
    # Оставляем в тесте только те SKU, которые есть в трейне
    common_sku = sku_train.intersection(sku_test)
    X_test = X_test[X_test['SKU'].isin(common_sku)]
    y_test = y_test[X_test.index]
    
    return X_train, y_train, X_test, y_test

In [6]:
def create_column_transformer(X_train_features):
    numeric_cols = X_train_features.select_dtypes(
        include=['int64', 'float64', 'int32', 'float32']
    ).columns.tolist()
    
    categorical_cols = X_train_features.select_dtypes(
        include=['object', 'category', 'bool']
    ).columns.tolist()

    #Создание трансформера
    transformers = []
    
    if numeric_cols:
        transformers.append(
            ('num', StandardScaler(), numeric_cols)
        )
    
    if categorical_cols:
        transformers.append(
            ('cat', OrdinalEncoder(
                handle_unknown='use_encoded_value',
                unknown_value=-1
            ), categorical_cols)
        )
    
    preprocessor = ColumnTransformer(
        transformers=transformers,
        remainder='passthrough',  
        n_jobs=-1
    )
    
    return preprocessor, numeric_cols, categorical_cols

In [7]:
low_sales_model = XGBRegressor(    
    reg_lambda=0.5,
    reg_alpha=0.5,
    
    max_depth=6,      
    subsample=0.8,      

    learning_rate=0.05, 
    n_estimators=1000,

    enable_categorical=True,
    
    random_state=42
)

medium_sales_model = XGBRegressor(    
    reg_lambda=0.5,
    reg_alpha=0.5,
    
    max_depth=6,      
    subsample=0.8,      

    learning_rate=0.05, 
    n_estimators=1000,

    enable_categorical=True,
    
    random_state=42
)

high_sales_model = XGBRegressor(    
    reg_lambda=1,
    reg_alpha=0.5,
    
    max_depth=6,      
    subsample=0.8,      

    learning_rate=0.05, 
    n_estimators=1000,

    enable_categorical=True,
    
    random_state=42
)

In [8]:
def mase(y_true, y_pred, y_train):
    naive_forecast = np.roll(y_train, 1)[1:]  # прогноз = предыдущее значение
    mae_naive = mean_absolute_error(y_train[1:], naive_forecast)
    mae_model = mean_absolute_error(y_true, y_pred)
    return mae_model / mae_naive

In [9]:
X_train, y_train, X_test, y_test = data_preprocessing(low_sales, test_size=0.2, target_col='Value')

feature_engineer = FeatureEngineer(date_col='Date', sku_col='SKU', target_col='Value')
feature_engineer.fit(X_train, y_train)

X_train_features = feature_engineer.transform(X_train, y_train)
X_test_features = feature_engineer.transform(X_test, y_test)

preprocessor, num_cols, cat_cols = create_column_transformer(X_train_features)

low_sales_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', low_sales_model)  
])


In [10]:
low_sales_pipeline.fit(X_train_features, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(n_jobs=-1, remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['index', 'lag_1', 'lag_2',
                                                   'lag_3', 'lag_4', 'lag_5',
                                                   'lag_6', 'lag_7', 'lag_14',
                                                   'lag_21', 'lag_28', 'lag_30',
                                                   'lag_60', 'lag_90', 'diff_1',
                                                   'diff_7', 'diff_14',
                                                   'diff_28', 'pct_change_1',
                                                   'pct_change_7',
                                                   'pct_change_28',
                                                   'rolling_mean_3',
                                                   'rolling...
                              feature_types=None, feature_weights=None,
                              gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.05,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=6, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=1000, n_jobs=None,
                              num_parallel_tree=None, ...))])

In [11]:
y_pred = low_sales_pipeline.predict(X_test_features)

print('Метрики качества:')
print('MAE: ', mean_absolute_error(y_pred, y_test))
print('MASE:', mase(y_test, y_pred, y_train))

Метрики качества:
MAE:  0.059304108240100385
MASE: 0.03920986732091605


In [15]:
model_package = {
    'feature_engineer_class': FeatureEngineer, 
    'feature_engineer_instance': feature_engineer,
    'pipeline': low_sales_pipeline,
    'metadata': {
        'date_col': 'Date',
        'sku_col': 'SKU', 
        'target_col': 'Value',
        'model_type': 'XGBoost'
 }
}   

In [16]:
import dill

with open('model_dill.pkl', 'wb') as f:
    dill.dump(model_package, f)